In [1]:
import numpy as np
import pandas as pd
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
from itertools import combinations
pd.set_option('display.max_columns', 20000)
import datetime
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#To read data from SQl database
from psycopg2 import connect
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
import pandas.io.sql as psql

Steps:
1. Create data base in postgres with name metis_project3
2. To open terminal and run "psql -d create_db.sql" 

create_db.sql is in SQL_DB folder

In [3]:
params = {
    'host': '127.0.0.1',
    'user': 'natali',
    'port': 5432
}

In [4]:
connection = connect(**params, dbname='metis_project3')

In [5]:
data = pd.read_sql('SELECT * FROM public.data', connection)
eq = pd.read_sql('SELECT * FROM public.equipment', connection)
positive_labels = pd.read_sql('SELECT * FROM public.fact', connection)
event = pd.read_sql('SELECT * FROM public.event_type', connection)

Alternatively load the data directly from csv 

In [6]:
eq = pd.read_csv('data/given/equipment.csv', sep = ',"')
data = pd.read_csv('data/given/data.csv', sep = ';')
positive_labels = pd.read_csv('data/given/facts.csv', sep = ",")

/Users/natali/anaconda3/envs/py37/lib/python3.6/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


Check if we have all data point found in positive labels in train set.   
We don't have any information about this number 158528852857, 158530004641 in the train data, however it is in positive labels.

In [7]:
to_delete = []
for i, row in enumerate(positive_labels.values):
    if len(data[data['msisdn'] == row[0]]) == 0:
        to_delete.append(row[0])
    if len(data[data['msisdn'] == row[1]]) == 0:
        to_delete.append(row[1])

In [8]:
to_delete

[158528852857, 158530004641]

In [9]:
positive_labels = positive_labels.drop(positive_labels[positive_labels.iloc[:,1].isin(to_delete)].index[0], axis = 0).reset_index(drop = True)
positive_labels = positive_labels.drop(positive_labels[positive_labels.iloc[:,1].isin(to_delete)].index[0], axis = 0).reset_index(drop = True)

Prettify eq file

In [10]:
eq.columns = ['tac', 'vendor', 'platform', 'type']
eq['tac'] = eq['tac'].apply(lambda x: x[1:])
eq['vendor'] = eq['vendor'].apply(lambda x: x[1:-2])
eq['platform'] = eq['platform'].apply(lambda x: x[1:-2])
eq['type'] = eq['type'].apply(lambda x: x[1:-3])

In [11]:
eq.head()

,tac,vendor,platform,type
0,"""00100429""""",PROTO,Proprietary,Phon
1,"""00100430""""",PROTO,Proprietary,Phon
2,"""00100431""""",PROTO,Proprietary,Phon
3,"""00100432""""",PROTO,Proprietary,Phon
4,"""00100433""""",PROTO,Proprietary,Phon


In [12]:
data.head()

,lac,cid,msisdn,imei,event_type,tstamp,long,lat,max_dist,cell_type,start_angle,end_angle
0,5029,40798,158529599791,3.531111e+14,0,1369252800974,37.9305,55.7815,2101,OUTDOOR,220.0,280.0
1,7782,56870,158520145943,1.292900e+13,5,1369252801396,37.4239,55.7426,2,OUTDOOR,45.0,105.0
2,7794,32226,158521798391,3.519940e+14,5,1369252802013,37.7034,55.5803,1125,OUTDOOR,0.0,60.0
3,7758,33528,158537830573,3.586270e+14,0,1369252802331,37.7895,55.7042,1158,OUTDOOR,210.0,270.0
4,5060,17568,158510204039,3.524581e+14,8,1369252803586,37.1900,55.8308,500,OUTDOOR,180.0,240.0


In [13]:
print("How many observations in our data set? -  {}".format(len(data)))
print("How many unique numbers (msisdn) in our data set? - {}".format(data['msisdn'].nunique()))

How many observations in our data set? -  174123
How many unique numbers (msisdn) in our data set? - 2361


It's computationally expensive to make all possible pair on current train data. Let's try to make data set smaller (include only data points which in positive_labels file and make all possible intersations between them) and see what result we can get from smaller data set.

In [14]:
list_of_positive_labels = []
for j in range(2):
    for i in range(len(positive_labels)):
        list_of_positive_labels.append(positive_labels.iloc[i, j])

In [15]:
data_small = pd.DataFrame(columns = data.columns)
for i in list(data['msisdn'].unique()):
    if i in list_of_positive_labels:
        data_small = data_small.append(data[data['msisdn'] == i])
data_small = data_small.reset_index(drop = True)

In [16]:
data_small.head()

,lac,cid,msisdn,imei,event_type,tstamp,long,lat,max_dist,cell_type,start_angle,end_angle
0,7798,3866,158528850493,NaN,0,1369252834051,37.6735,55.7363,403,OUTDOOR,255.0,315.0
1,7761,15621,158528850493,NaN,0,1369253132222,37.6665,55.7385,337,OUTDOOR,170.0,230.0
2,7798,3866,158528850493,1.300201e+13,3,1369253132299,37.6735,55.7363,403,OUTDOOR,255.0,315.0
3,7798,3866,158528850493,NaN,0,1369253152222,37.6735,55.7363,403,OUTDOOR,255.0,315.0
4,7761,15621,158528850493,1.300201e+13,3,1369264234644,37.6665,55.7385,337,OUTDOOR,170.0,230.0


"Msisdn" column is how many milliseconds passed from 1970-01-01. The next function convert this column in ordinary date-time format

In [17]:
data_small['tstamp'] = data_small['tstamp'].apply(lambda x: datetime.datetime(1970, 1, 1, 0, 0) + datetime.timedelta(milliseconds = x))

In [18]:
data_small.tstamp = data_small.tstamp.apply(lambda x: x.replace(second=0, microsecond=0))

In [19]:
data_small.to_csv('data/data_small.csv')

Check if a person with two sim-card can have only one phone number (imei)

In [20]:
positive_labels;

Since cid - identificator of base station unique in terms of lac the folowing cell creates columns 'lac_cid' as a combination of cid and lac and this gives as unique location for each station

In [21]:
data_small['lac_cid'] = data_small['lac'].astype(str)  + data_small['cid'].astype(str)

Grouping by msisdn. The first column of result dataframe msisdn itself, the second column - list of all locations where this number was registered

In [22]:
group = data_small.groupby(['msisdn'], as_index=False).agg({'lac_cid':(lambda x: list(x))}).rename(columns={'msisdn':'number', 'lac_cid':'set_'})

Creating all posible combinations of msisdn

In [23]:
df = pd.DataFrame(columns = ['number_1', 'set_1', 'number_2', 'set_2'])

for index in list(combinations(group.index, 2)):
    
    df0 = pd.DataFrame(group.loc[index[0]]).T.reset_index(drop=True)
    df1 = pd.DataFrame(group.loc[index[1]]).T.reset_index(drop=True)
    df_temp = pd.concat([df0, df1], axis = 1)
    df_temp.columns = ['number_1', 'set_1', 'number_2', 'set_2']
    df = df.append(df_temp)
df = df.set_index(pd.Index(range(len(df))))

The following fuction counts the intersection in locations between each possible pairs of numbers (how many times in persentage they were at the same locations) 

In [24]:
import copy 
def intersect_loc(x):
    list1 = x['set_1']
    list2 = x['set_2']
    
    min_len = min(len(list1), len(list2))
    res = []
    list_1 = list1.copy()
    for i in list_1:
        if i in list2:
            res.append(i)
            list1.remove(i)
            list2.remove(i)
    return len(res)/(min_len+1)

In [25]:
df['intersect_loc'] = df.apply(intersect_loc, axis =1)

The following function returns the dictionary of possible numbers which belong to the same person. It chooses candidates by selecting maximum intersection between them

In [26]:
def create_key():
    out = {}
    
    for i in list(data_small['msisdn'].unique()):
        out[i] = data_small[data_small['msisdn'] == i]
    return out

In [27]:
dict_number = create_key()

The following fuction counts how many times between each possible pair of numbers in dataset following event occured: these numbers happened to be at the same location (in terms of the same group of cell towers) in the same time (within 15 minutes) 

In [28]:
def intersectin_time_loc(x):
    
    margin = datetime.timedelta(minutes=15)
    index_1 = x['number_1']
    index_2 = x['number_2']
    
    counter = 0
    set_1 = dict_number[index_1]
    set_2 = dict_number[index_2]

    for i in set_1['tstamp']:
        for j in set_2['tstamp']:
            if i - margin <= j <= i + margin:
                if set_1[set_1['tstamp'] == i]['lac'].iloc[0] == set_2[set_2['tstamp'] == j]['lac'].iloc[0]:
                    counter += 1
    return counter

In [29]:
%time df['time+loc_15'] = df.apply(intersectin_time_loc, axis = 1)

CPU times: user 7min 44s, sys: 794 ms, total: 7min 44s
Wall time: 7min 44s


In [30]:
df = df.drop(['set_1', 'set_2'], axis = 1)

In [31]:
target = []

for entry in df[['number_1', 'number_2']].values:
    for pair in positive_labels.values:
        add=0
        if entry[0] in pair and entry[1] in pair:
            target.append(1)
            add=1
            break
    if add==0:
        target.append(0)

In [32]:
df['target'] = target

The following fuction counts how many times between each possible pair of numbers in dataset following event occured: these numbers happened to be at the same location (in terms of the same cell tower) in the same time (within 10 minutes) and cell tower registered signals from two numbers from the same angles (+/-30 degrees) 

In [33]:
def intersectin_time_loc_angle(x):
    
    margin_time = datetime.timedelta(minutes=10)
    margin_angle = 30
    index_1 = x['number_1']
    index_2 = x['number_2']
    
    counter = 0
    set_1 = dict_number[index_1]
    set_2 = dict_number[index_2]
    
    
    for i in set_1['tstamp']:
        for j in set_2['tstamp']:
            if i - margin_time <= j <= i + margin_time:
                tstamp_i = set_1[set_1['tstamp'] == i]
                tstamp_j = set_2[set_2['tstamp'] == j]
                
                lac_cid_i = tstamp_i['lac_cid'].iloc[0]
                lac_cid_j = tstamp_j['lac_cid'].iloc[0]
                
                if lac_cid_i == lac_cid_j:
                    
                    start_angle_i = tstamp_i['start_angle'].iloc[0]
                    start_angle_j = tstamp_j['start_angle'].iloc[0]
                    
                    if start_angle_i - margin_angle <= start_angle_j <= start_angle_i + margin_angle:
                        counter += 1
                        
    return counter

In [34]:
%time df['time+log+angle'] = df.apply(intersectin_time_loc_angle, axis = 1)

CPU times: user 6min 23s, sys: 771 ms, total: 6min 24s
Wall time: 10min 44s


Save dataframe fore modeling

In [35]:
df.to_csv('data/df.csv')